In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb


In [3]:
df = pd.read_csv("data/vehicles.csv")

In [4]:
num_row = len(df)
print("Number of rows: {}".format(num_row))

Number of rows: 426880


In [5]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


In [6]:
df.describe()

,id,price,year,odometer,county,lat,long
count,4.268800e+05,4.268800e+05,425675.000000,4.224800e+05,0.0,420331.000000,420331.000000
mean,7.311487e+09,7.519903e+04,2011.235191,9.804333e+04,NaN,38.493940,-94.748599
std,4.473170e+06,1.218228e+07,9.452120,2.138815e+05,NaN,5.841533,18.365462
min,7.207408e+09,0.000000e+00,1900.000000,0.000000e+00,NaN,-84.122245,-159.827728
25%,7.308143e+09,5.900000e+03,2008.000000,3.770400e+04,NaN,34.601900,-111.939847
50%,7.312621e+09,1.395000e+04,2013.000000,8.554800e+04,NaN,39.150100,-88.432600
75%,7.315254e+09,2.648575e+04,2017.000000,1.335425e+05,NaN,42.398900,-80.832039
max,7.317101e+09,3.736929e+09,2022.000000,1.000000e+07,NaN,82.390818,173.885502


# Predict used car sales price from its information

## EDA

In [7]:
# Filter out feature with too many nan value
nan_val = df.isna().sum()

def na_filter(na, threshold = 0.4): # only select variables that passees the threshold
    col_pass = []
    for i in na.keys():
        if na[i] / num_row < threshold:
            col_pass.append(i)
    return col_pass
col_keep = na_filter(nan_val)
df_cleaned = df[col_keep]
df_cleaned.columns
print("Columns to remove: {}".format(set(df.columns) - set(col_keep)))

Columns to remove: {'cylinders', 'condition', 'size', 'county'}


In [8]:
eda_disc = ['manufacturer','model', 'condition', 
       'cylinders', 'fuel', 'title_status', 'transmission', 'drive', 
       'size', 'type', 'paint_color', 'county', 'state']
# dist_cols = []
eda_continuous = ['price', 'year', 'odometer', 'posting_date']

In [9]:
for col in eda_disc:
    print("Distribution of {} col".format(col))
    print(df[col].value_counts()[:10])
    print()

Distribution of manufacturer col
ford         70985
chevrolet    55064
toyota       34202
honda        21269
nissan       19067
jeep         19014
ram          18342
gmc          16785
bmw          14699
dodge        13707
Name: manufacturer, dtype: int64

Distribution of model col
f-150             8009
silverado 1500    5140
1500              4211
camry             3135
silverado         3023
accord            2969
wrangler          2848
civic             2799
altima            2779
escape            2746
Name: model, dtype: int64

Distribution of condition col
good         121456
excellent    101467
like new      21178
fair           6769
new            1305
salvage         601
Name: condition, dtype: int64

Distribution of cylinders col
6 cylinders     94169
4 cylinders     77642
8 cylinders     72062
5 cylinders      1712
10 cylinders     1455
other            1298
3 cylinders       655
12 cylinders      209
Name: cylinders, dtype: int64

Distribution of fuel col
gas         35620

In [10]:
(df.price > 80000).sum()

1627

In [11]:
(df.year < 2000).sum()

25250

In [12]:
(df.odometer > 200000).sum()

23170

In [13]:
# Remove problematic data
to_remove = []
to_remove.extend(df.index[df.price > 80000].tolist())
to_remove.extend(df.index[df.year < 2000].tolist())
to_remove.extend(df.index[df.odometer > 200000].tolist())

df_cleaned = df_cleaned.iloc[~df.index.isin(to_remove), :]


In [14]:
print("Number of data after cleaning: {}".format(len(df_cleaned)))

Number of data after cleaning: 380684


## Data preprocessing

In [1]:
numerics = set(['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64'])
categorical_columns = []
features = df_cleaned.columns.values.tolist()
for col in features:
    if df_cleaned[col].dtype in numerics: 
        continue
    categorical_columns.append(col)

# Encoding categorical features
for col in categorical_columns[:1]:
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(list(df_cleaned[col].astype(str).values))
    df_cleaned[col] = enc.transform(list(df_cleaned[col].astype(str).values))

NameError: name 'df_cleaned' is not defined

In [57]:
X = df_cleaned.iloc[:, df_cleaned.columns != 'price']
y = df_cleaned.iloc[:, df_cleaned.columns == 'price']

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [59]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266478 entries, 219088 to 136985
Data columns (total 21 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            266478 non-null  int64  
 1   url           266478 non-null  object 
 2   region        266478 non-null  object 
 3   region_url    266478 non-null  object 
 4   year          265643 non-null  float64
 5   manufacturer  257711 non-null  object 
 6   model         264205 non-null  object 
 7   fuel          264437 non-null  object 
 8   odometer      263456 non-null  float64
 9   title_status  260981 non-null  object 
 10  transmission  264820 non-null  object 
 11  VIN           177239 non-null  object 
 12  drive         185314 non-null  object 
 13  type          212794 non-null  object 
 14  paint_color   186436 non-null  object 
 15  image_url     266431 non-null  object 
 16  description   266429 non-null  object 
 17  state         266478 non-null  object 
 18 

## Model fitting

### Linear Regression

In [61]:
def validation(model, X_train, y_train, X_test, y_test):
    ypred_train = model.predict(X_train)  
    ypred_test = model.predict(X_test)

    print("train_ytrue = ", y_train[:5].values)
    print("train_ypred = ", ypred_train[:5])

    train_r2 = r2_score(y_train, ypred_train) * 100
    print("train_r2 = {:.4f}".format(train_r2))   

    train_mse = mean_squared_error(y_train, ypred_train) ** 0.5 * 100
    print('train_mse = {:.4f}'.format(train_mse))  

    print("test_ytrue = ", y_test[:5].values)
    print("test_ypred = ", ypred_test[:5])
    
    test_r2 = r2_score(y_test, ypred_test) * 100
    print("test_r2 = {:.4f}".format(test_r2))   

    test_mse = mean_squared_error(y_test, ypred_test) ** 0.5 * 100
    print('test_mse = {:.4f}'.format(test_mse))  


In [62]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
validation(linreg, X_train, y_train, X_test, y_test)

ValueError: could not convert string to float: 'https://northmiss.craigslist.org/cto/d/columbus-mazda-b3000/7306622276.html'